In [ ]:
# Import

from loan_utils_v2 import Loan

In [4]:
c1 = 100000 # Capital
r1 = 0.0300 # Annual Interest rate
fper1 = 12 # Number of periodic payments (number of months per year)
per1 = 25 # Number of years 
n1 = 300 # Specific period 

In [6]:
loan = Loan(capital= c1, annual_rate= r1, years= per1, periods_per_year=fper1)
# dir(loan)
loan.get_basic_info()
print(30*'-')
loan.get_period_for_targeted_cumulated_interest(threshold=0.5)
print(30*'-')
loan.get_situation_at_period(period=300)

Capital :  100000
Annual rate :  0.03
Number of years :  25
Number of periods per year :  12
Total number of payments :  300
Periodic rate :  0.0025
Periodic payment :  -474.21131385767984
Total interest paid : 
------------------------------
Period :  96
------------------------------
Situation of loan at period :  300
Paid capital :  100000.0
Paid interest :  42263.39
cumulated Payment at period :  142263.39
Remaining Period :  0
Remaining capital : 0.0
Remaining interest : 0.0
Remaining payment : 0.0
total capital : 100000.0


In [7]:
loan.formated_depreciation_table()

╒════════════╤══════════════╤════════════╤════════════╤═══════════╤════════════╤═══════════╤════════════╕
│   Echéance │ Mensualité   │ Capital    │ Intérêts   │ Capital   │ Intérêts   │ Capital   │ Intérêts   │
│            │ échéance     │ échéance   │ échéance   │ cumulé    │ cumulé     │ restant   │ restants   │
╞════════════╪══════════════╪════════════╪════════════╪═══════════╪════════════╪═══════════╪════════════╡
│          1 │ 474,21       │ 224,77     │ 249,44     │ 448,98    │ 499,44     │ 99551,02  │ 41763,95   │
├────────────┼──────────────┼────────────┼────────────┼───────────┼────────────┼───────────┼────────────┤
│          2 │ 474,21       │ 225,33     │ 248,88     │ 674,32    │ 748,32     │ 99325,68  │ 41515,08   │
├────────────┼──────────────┼────────────┼────────────┼───────────┼────────────┼───────────┼────────────┤
│          3 │ 474,21       │ 225,90     │ 248,31     │ 900,21    │ 996,63     │ 99099,79  │ 41266,76   │
├────────────┼──────────────┼────────────┼────

'End'